# Lewensztain
Jakub Karbowski

In [19]:
import Pkg
Pkg.add([
    "WordTokenizers",
])

using Random: seed!
using WordTokenizers: tokenize

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


# Edit distance
Algorytm dynamiczny.

Złożoność: $O(|x| \cdot |y|)$.

Pamięć: $O(|x| \cdot |y|)$.

Zmiany oznaczane są następująco:

- ```[-a]``` usunięcie 'a'

- ```[+a]``` dodanie 'a'

- ```[a->b]``` zamiana 'a' na 'b'

In [20]:
function edit(t1, t2; visualize=true, delta=((x, y) -> x == y ? 0 : 1))
    UP = 1
    LEFT = 2
    UPLEFT = 3
    
    n1 = length(t1)
    n2 = length(t2)

    f = zeros(Int, n1 + 1, n2 + 1)
    f[1:end, 1] = 0:n1
    f[1, 1:end] = 0:n2
    
    parents = zero(f)
    parents[1:end, 1] .= UP
    parents[1, 1:end] .= LEFT
    parents[1, 1] = 0
    
    for i = 2:n1+1, j = 2:n2+1
        up = f[i-1, j] + 1
        left = f[i, j-1] + 1
        upleft = f[i-1, j-1] + delta(t1[i-1], t2[j-1])

        f[i, j] = min(up, left, upleft)
        
        # pamiętanie ścieżki w grafie
        parents[i, j] =
            if f[i, j] == up
                UP
            elseif f[i, j] == left
                LEFT
            else
                UPLEFT
            end
    end
    
    function backtrack(i, j)
        if parents[i, j] == UP
            backtrack(i - 1, j)
            println("$(t2[1:j-1])[-$(t1[i-1])]$(t1[i:end])")
        elseif parents[i, j] == LEFT
            backtrack(i, j - 1)
            println("$(t2[1:j-2])[+$(t2[j-1])]$(t1[i:end])")
        elseif parents[i, j] == UPLEFT
            backtrack(i - 1, j - 1)
            if f[i, j] != f[i - 1, j - 1]
                println("$(t2[1:j-2])[$(t1[i-1])->$(t2[j-1])]$(t1[i:end])")
            end
        end
    end
    
    visualize && backtrack(n1+1, n2+1)
    
    f[end, end]
end

edit("levenshtein", "lewensztain")

le[v->w]enshtein
lewens[h->z]tein
lewenszt[e->a]in


3

In [21]:
edit("los", "kloc")

[+k]los
klo[s->c]


2

In [22]:
edit("kwintesencja", "quintessence")

[k->q]wintesencja
q[w->u]intesencja
quintes[+s]encja
quintessenc[j->e]a
quintessence[-a]


5

In [23]:
edit("ATGAATCTTACCGCCTCG", "ATGAGGCTCTGGCCCCTG")

ATGA[A->G]TCTTACCGCCTCG
ATGAG[T->G]CTTACCGCCTCG
ATGAGGCT[+C]TACCGCCTCG
ATGAGGCTCT[A->G]CCGCCTCG
ATGAGGCTCTG[+G]CCGCCTCG
ATGAGGCTCTGGCC[-G]CCTCG
ATGAGGCTCTGGCCCCT[-C]G


7

# Podciąg

In [24]:
lcs(x, y) = (
    length(x) + 
    length(y) -
    edit(x, y,
         visualize=false,
         delta=(x, y) -> x == y ? 0 : 2)
    ) / 2

lcs("cbabac", "abcabba")

4.0

In [25]:
text = join(readlines("data/romeo-i-julia-700.txt"), '\n')

# podział na słowa
toksfull = tokenize(text)
seed!(42)
toks1 = [t for t=toksfull if rand(0:100) >= 3]
toks2 = [t for t=toksfull if rand(0:100) >= 3]

println("All tokens = $(length(toksfull))")
println("Tokens 1   = $(length(toks1))")
println("Tokens 2   = $(length(toks2))")

lcs(toks1, toks2)

All tokens = 2191
Tokens 1   = 2128
Tokens 2   = 2115


2054.0

# Diff
Stworzone narzędzie jest prawie identyczne do prawdziwego ```diff```:

- ```NaM``` dodanie linii M z pliku 2 za linię N w pliku 1

- ```NdM``` linia N w pliku 1 została usunięta, kursor w pliku 2 jest na linii M

- ```NcM``` zamiana linii N w pliku 1 na linię M z pliku 2

Jedyna różnica to brak grupowania sąsiednich operacji.

In [26]:
function diff(f1, f2)
    delta(x, y) = x == y ? 0 : 1
    
    t1 = readlines(f1)
    t2 = readlines(f2)
    
    # kod skopiowany z funkcji edit,
    # można wyciągnąć część wspólną
    # ale wolę już nie psuć
    
    UP = 1
    LEFT = 2
    UPLEFT = 3
    
    n1 = length(t1)
    n2 = length(t2)

    f = zeros(Int, n1 + 1, n2 + 1)
    f[1:end, 1] = 0:n1
    f[1, 1:end] = 0:n2
    
    parents = zero(f)
    parents[1:end, 1] .= UP
    parents[1, 1:end] .= LEFT
    parents[1, 1] = 0
    
    for i = 2:n1+1, j = 2:n2+1
        up = f[i-1, j] + 1
        left = f[i, j-1] + 1
        upleft = f[i-1, j-1] + delta(t1[i-1], t2[j-1])

        f[i, j] = min(up, left, upleft)
        
        parents[i, j] =
            if f[i, j] == up
                UP
            elseif f[i, j] == left
                LEFT
            else
                UPLEFT
            end
    end
    
    # jedyna różnica, inne printy
    function backtrack(i, j)
        if parents[i, j] == UP
            backtrack(i - 1, j)
            println("$(i-1)d$(j-1)")
            println("< $(t1[i-1])")
        elseif parents[i, j] == LEFT
            backtrack(i, j - 1)
            println("$(i-1)a$(j-1)")
            println("> $(t2[j-1])")
        elseif parents[i, j] == UPLEFT
            backtrack(i - 1, j - 1)
            if f[i, j] != f[i - 1, j - 1]
                println("$(i-1)c$(j-1)")
                println("< $(t1[i-1])")
                println("---")
                println("> $(t2[j-1])")
            end
        end
    end
    
    backtrack(n1+1, n2+1)
end

diff("data/diff1", "data/diff2")

1a2
>                                     USTAWA
3c4
<          o zryczałtowanym podatku dochodowym od niektórych przychodów
---
>                         osiąganych przez osoby fizyczne
11d11
<   gospodarczej opłacają zryczałtowany podatek dochodowy w formie:
12a13
>    2) karty podatkowej.
15c16
<   ewidencjonowanych oraz zryczałtowanego podatku dochodowego od przychodów
---
> 3. Wpływy z podatku dochodowego opłacanego w formie ryczałtu od przychodów
15a17
>   osób duchownych stanowią dochód budżetu państwa.


## Prawdziwy diff:
```
1a2
>                                     USTAWA
3c4
<          o zryczałtowanym podatku dochodowym od niektórych przychodów
---
>                         osiąganych przez osoby fizyczne
11d11
<   gospodarczej opłacają zryczałtowany podatek dochodowy w formie:
12a13
>    2) karty podatkowej.
15c16,17                    <- tutaj widać różnicę (grupowanie)
<   ewidencjonowanych oraz zryczałtowanego podatku dochodowego od przychodów
---
> 3. Wpływy z podatku dochodowego opłacanego w formie ryczałtu od przychodów
>   osób duchownych stanowią dochód budżetu państwa.
```

In [33]:
lines = readlines("data/romeo-i-julia-700.txt")
seed!(42)
lines1 = [join([t for t=tokenize(l) if rand(1:1000) != 1], ' ') for l=lines if rand(1:500) != 1]
lines2 = [join([t for t=tokenize(l) if rand(1:1000) != 1], ' ') for l=lines if rand(1:500) != 1]

write("data/rom1", join(lines1, '\n'))
write("data/rom2", join(lines2, '\n'))

diff("data/rom1", "data/rom2")

33c33
< * Obywatele weroneńscy , różne osoby płci obojej , liczący się do przyjaciół obu domów , maski , straż wojskowa i inne osoby .
---
> * Obywatele weroneńscy , różne osoby płci obojej , liczący się przyjaciół obu domów , maski , straż wojskowa i inne osoby .
58c58
< Aż ją dzieci zgon przedwczesny ,
---
> Aż ją zakończy dzieci zgon przedwczesny ,
77d76
< 
115a115
> SAMSON
152c152
< Tym lepiej , że się liczysz do zwierząt ; bo gdybyś się liczył do ryb , to byłbyś pewnie sztokfiszem. Weź no się za instrument , bo oto nadchodzi dwóch domowników Montekiego .
---
> Tym lepiej , że się liczysz do zwierząt ; bo gdybyś się liczył do ryb , to byłbyś pewnie sztokfiszem. Weź no się za , bo oto nadchodzi dwóch domowników Montekiego .
286a287
> Do mnie , Benwolio ! pilnuj swego życia .
368c369
< Wzniecicie kiedyś podobną ,
---
> Wzniecicie kiedyś waśń podobną ,
452a454
> BENWOLIO
568a571
> Nad czym , poczciwa duszo ?
622c625
< Tylko mi klucz dać do tego problemu ,
---
> Tylko mi dać do tego pr

## Prawdziwy diff:
```
33c33
< * Obywatele weroneńscy , różne osoby płci obojej , liczący się do przyjaciół obu domów , maski , straż wojskowa i inne osoby .
---
> * Obywatele weroneńscy , różne osoby płci obojej , liczący się przyjaciół obu domów , maski , straż wojskowa i inne osoby .
58c58
< Aż ją dzieci zgon przedwczesny ,
---
> Aż ją zakończy dzieci zgon przedwczesny ,
77d76
<
115a115
> SAMSON
152c152
< Tym lepiej , że się liczysz do zwierząt ; bo gdybyś się liczył do ryb , to byłbyś pewnie sztokfiszem. Weź no się za instrument , bo oto nadchodzi dwóch domowników Montekiego .
---
> Tym lepiej , że się liczysz do zwierząt ; bo gdybyś się liczył do ryb , to byłbyś pewnie sztokfiszem. Weź no się za , bo oto nadchodzi dwóch domowników Montekiego .
286a287
> Do mnie , Benwolio ! pilnuj swego życia .
368c369
< Wzniecicie kiedyś podobną ,
---
> Wzniecicie kiedyś waśń podobną ,
452a454
> BENWOLIO
568a571
> Nad czym , poczciwa duszo ?
622c625
< Tylko mi klucz dać do tego problemu ,
---
> Tylko mi dać do tego problemu ,
```

# Wnioski (rezultaty?)

Bardzo ładnie pokazywany jest ciąg operacji
do zamiany jednego tekstu na drugi (funkcja ```edit```).

Stworzone narzędzie działa zgodnie z prawdziwym ```diff```.
Zaskakujące jest jak dobrze to działa.